## FCS Lab 2 Submission Report

* Name of Student(s): Leong Yun Qin Melody
* Student ID(s): 1004489

In [ ]:
import base64
import requests
def XOR(a, b):
    r = b""
    for x, y in zip(a, b):
        r += (x ^ y).to_bytes(1, "big")
    return r
class Client:
    def __init__(self, endpoint, uid):
        self.endpoint = endpoint
        self.uid = str(uid).lower().strip()
    def post(self, url, data=None):
        r = requests.post(url, json=data).json()
        if not r["success"]:
            print("Warning: something might be wrong with the server")
            print("If you don't think is your mistake, please report it!")
        return r
    def get_story_cipher(self):
        url = self.endpoint+"/story"
        return requests.get(url).json()
    def post_story_plaintext(self, solution):
        url = self.endpoint+"/story"
        solution = str(solution).lower().strip()
        data = {"solution":solution}
        return self.post(url, data)
    def get_score_msg_cipher(self):
        url = self.endpoint+"/score"
        data = {"request":"get_msg", "id":self.uid}
        return self.post(url, data)
    def submit_score_msg_cipher(self, cipher_base64):
        url = self.endpoint+"/score"
        data = {"request":"decrypt_msg", "id":self.uid, "cipher": cipher_base64}
        return self.post(url, data)
    def base64_encode_bytes(self, byte_array):
        return str(base64.b64encode(byte_array))[2:-1]
    def base64_decode_bytes(self, base64_string):
        return base64.b64decode(base64_string)

In [ ]:
endpoint = "http://35.197.130.121/"
uid = "1004489"

client = Client(endpoint, uid)

## Part I: Story - Substitution Cipher

1. GET the cipher for the story
2. Crack this with frequency analysis
3. POST it back to the server to check (example is provided below)

If the response contains `'solution_correct': 'correct'`, then your decryption is correct. Otherwise, a distance will be provided to let you know how far off you are. If you are off by a tiny bit (say, 1 or 2), you can check things like line-ending, extra space at start/end etc. The verification is not case sensitive.

In [ ]:
story_cipher = client.get_story_cipher()["cipher"]
print("story_cipher:", story_cipher[:50], "...")
#write to txt 

In [ ]:
#decrypt with frequency analysis
#count number of each type of letters (take the proportion)
def count_freq_letters(story_cipher):
    #dictionary to map out letter to freq count
    letter = {"A":0, "B":0, "C":0, "D":0,"E":0, "F":0,
                "G":0, "H":0, "I":0,"J":0,"K":0,"L":0,"M":0,
                "N":0,"P":0,"Q":0, "R":0,"S":0,"T":0,"U":0,"V":0, "W":0,"X":0,"Y":0,"Z":0}
    #loop through the string 
    for alphabet in story_cipher:
        if alphabet in letter.keys():
            letter[alphabet] += 1

    #find the letter with largest count
    Counter = 0
    for char, freq_count in letter.items():
        #compare and store
        if freq_count > Counter:
            max_cipher_character = char
            Counter = freq_count
    #ensure the dict is correct 
    print(letter)
    return f"The letter appearing the most in the cipher text is {max_cipher_character}"

print(count_freq_letters(story_cipher))

#REPLACE the letter U with e       

In [ ]:
# replace the letter U with e
def replace_first_function(story_cipher):
    #make sure of original text 
    #print(f"{story_cipher} is the original cipher")
    return story_cipher.replace("U","e")
#print(replace_first_function(story_cipher))
    

In [ ]:
#replace the Y with I 
most_freq_with_e = replace_first_function(story_cipher)
def replace_single_letter1(most_freq_with_e=most_freq_with_e):
    single_1 = most_freq_with_e.replace("Y","i")
    return single_1

#test first
#text = replace_first_function(story_cipher)
#text = replace_single_letter1()
#print(replace_single_letter1())


In [ ]:
#try Q with A
replaced_Y_with_I = replace_single_letter1()
def replace_Q_with_A(replace_single_letter1=replaced_Y_with_I):
    return replace_single_letter1.replace("Q","a")
#print(replace_Q_with_A( ))
text = replace_Q_with_A()


In [ ]:
#function for this please work 
def mapcipher_to_realtext(txt):
    ciph_to_real ={
        "A":"k",
        "B":"l",
        "C":"m",
        "D":"n",
        "E":"o",
        "F":"p",
        "H":"r",
        "I":"s",
        "J":"t",
        "K":"u",
        "L":"v",
        "M":"w",
        "N":"x",
        "O":"y",
        "P":"z",
        "R":"b",
        "S":"c",
        "T":"d",
        "V":"f", 
        "W":"g",
        "X":"h",
        "Z":"j"
    }
    for character in txt:
        if character in ciph_to_real.keys():
            txt =txt.replace(character,ciph_to_real[character])
    return txt

In [ ]:
#replace the single words with knowledge
text = mapcipher_to_realtext(text)          


In [ ]:
# example posting a string back to the server


client.post_story_plaintext(text)
# a distance is provided for you to check how close you are

In [ ]:
# you can also load solution from a text file
with open("./solution.txt", "r") as file:
    PART_1_SOLUTION = file.read()
part_1_result = client.post_story_plaintext(PART_1_SOLUTION)
print(part_1_result)
assert part_1_result["solution_correct"] == "correct"

## Part II: Changing the Score Message - OTP

In [739]:
response = client.get_score_msg_cipher()
print(response)

{'cipher': 'KbkbJNdIrSl2Qum1htdDdaVeyKWBGd3nkldyFV7h1vOP7FtuF5UK4YJ+tL4=', 'hint': 'it is a OTP, you will not be able to guess it, find a way to edit the message without the OTP key', 'success': True}


In [740]:
cipher = client.base64_decode_bytes(response["cipher"])
print(cipher)
#convert the message to bytes

b')\xb9\x1b$\xd7H\xad)vB\xe9\xb5\x86\xd7Cu\xa5^\xc8\xa5\x81\x19\xdd\xe7\x92Wr\x15^\xe1\xd6\xf3\x8f\xec[n\x17\x95\n\xe1\x82~\xb4\xbe'


In [741]:
encoded_cipher = client.base64_encode_bytes(cipher)
response = client.submit_score_msg_cipher(encoded_cipher)

In [737]:
#new_cipher = hax()
#encoded_new_cipher = client.base64_encode_bytes(new_cipher)
#client.submit_score_msg_cipher(encoded_new_cipher)